# How DCT and DST are Related to the DFT

The math can be confusing, and the [scipy docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.dst.html) can be especially confusing, so it's worth seeing **in code** *exactly* how the DCT and the DST can be formed out of a DFT (FFT). This gives a solid picture of what exactly $\vec{Y}_{\text{long}}$ and $\vec{Y}_{\text{short}}$ are and how they're related, which is important for their usage in this package.

This is especially important if we start using [different variants](https://github.com/pavelkomarov/spectral-derivatives/blob/main/notebooks/dct_types.ipynb) due to the DCT-II's apparent better [energy compaction](https://dsp.stackexchange.com/questions/96172/why-does-the-dct-ii-have-better-energy-compaction-than-dct-i).

In [16]:
import numpy as np
from scipy.fft import dct, idct, dst, idst, fft, ifft

First some vectors to transform. We'll use a real one and an imaginary one, pretending they correspond to some even or odd derivative, respectively.

In [77]:
Y_re = np.array([3, 7, -1, 4]) # purely real
Y_im = Y_re*1j # make it purely imaginary

## DCT-I

$$\begin{align}
\text{DCT-I}([Y_0, Y_1, ... Y_N]) &= \text{FFT}([Y_0, Y_1, ... Y_{N-1}, Y_N, Y_{N-1}, ... Y_1])[:\!N+1] \\
&= M \cdot \text{IFFT}([Y_0, Y_1, ... Y_{N-1}, Y_N, Y_{N-1}, ... Y_{1}])[:\!N+1]\\
&= M \cdot \text{IDCT-I}([Y_0, Y_1, ... Y_N])
\end{align}
$$

The factor of $M$ appears because the inverse transform is normalized by the length of the input, $M = 2N$ in this case, so we have to unnormalize. The input to both the forward and inverse transforms can be the same, because the equality of $Y_k = Y_{M-k}$ means it doesn't matter which one multiplies the positive versus the negative exponential ($e^{\pm jnk \frac{2\pi}{M}}$), which are flipped for forward versus inverse transform.

This case occurs both when transforming to frequency domain using DCT-I and when transforming back from frequency domain for even-order derivatives.

In [44]:
def dct_i_fft(Y):
	Y_ext = np.concatenate((Y, Y[-2:0:-1])) # even extension, ordering not sensitive like the odd case
	return fft(Y_ext)[:len(Y)]

def dct_i_ifft(Y):
	Y_ext = np.concatenate((Y, Y[-2:0:-1])) # even extension
	return len(Y_ext)*ifft(Y_ext)[:len(Y)] # unnormalize inverse transform by multiplying by M = signal length

def idct_i_ifft(Y):
    Y_ext = np.concatenate((Y, Y[-2:0:-1]))
    return ifft(Y_ext)[:len(Y)]

assert np.all(np.abs(dct(Y_re, 1) - dct_i_fft(Y_re)) < 1e-10)
assert np.all(np.abs(dct(Y_re, 1) - dct_i_ifft(Y_re)) < 1e-10)
assert np.all(np.abs(idct(Y_re, 1) - idct_i_ifft(Y_re)) < 1e-10)

## DST-I

$$\begin{align}
\text{DST-I}(j \cdot [Y_0, Y_1, ... Y_N]) &= \text{FFT}([0, -Y_0, -Y_1, ... -Y_{N-1}, -Y_N, 0, Y_N, Y_{N-1}, ... Y_1])[1\!:\!N+2] \\
&= M \cdot \text{IFFT}([0, Y_0, Y_1, ... Y_{N-1}, Y_N, 0, -Y_N, -Y_{N-1}, ... -Y_1])[1\!:\!N+2]\\
&= M \cdot \text{IDST-I}([Y_0, Y_1, ... Y_N])
\end{align}
$$

The factor of $M$ appears because the inverse transform is normalized by the length of the input, $M = 2(N+1)$ in this case, so we have to unnormalize. Notice the addition of $0$s at the $1^{st}$ and $\frac{M}{2}^{th}$ indices, which make the function odd. (It doesn't make sense to sine transform a not-odd thing, and the DST-I assumes the function is odd around whole-number indices ahead of and behind the current signal, which means there must be zero crossings there.) The input to forward and inverse transforms is *negated*, because $Y_k = -Y_{M-k}$, so it *does* matter which one multiplies the positive versus the negative exponential ($e^{\pm jnk \frac{2\pi}{M}}$), which are flipped for forward versus inverse transform.

This case occurs when transforming odd-order derivatives back from frequency domain.

In [45]:
def dst_i_fft(Y):
	Y_ext = np.concatenate(([0], -Y, [0], Y[::-1])) # odd extension, with sign flip to account for fft rather than ifft
	return fft(Y_ext)[1:1+len(Y)]

def dst_i_ifft(Y):
	Y_ext = np.concatenate(([0], Y, [0], -Y[::-1])) # odd extension, this time with expected signs
	return len(Y_ext)*ifft(Y_ext)[1:1+len(Y)]

def idst_i_ifft(Y):
    Y_ext = np.concatenate(([0], Y, [0], -Y[::-1]))
    return ifft(Y_ext)[1:1+len(Y)]

assert np.all(np.abs(dst(1j*Y_im, 1) - dst_i_fft(Y_im)) < 1e-10)
assert np.all(np.abs(dst(1j*Y_im, 1) - dst_i_ifft(Y_im)) < 1e-10)
assert np.all(np.abs(idst(1j*Y_im, 1) - idst_i_ifft(Y_im)) < 1e-10)

## DCT-II

$$\begin{align}
\text{DCT-II}([Y_0, Y_1, ... Y_N]) &= \text{FFT}([0, Y_0, 0, Y_1, 0, ... Y_N, 0, Y_N, 0, Y_{N-1}, ... 0, Y_1])[:\!N+1] \\
&= M \cdot \text{IFFT}([0, Y_0, 0, Y_1, 0, ... Y_N, 0, Y_N, 0, Y_{N-1}, ... 0, Y_1])[:\!N+1]
\end{align}
$$

For the DCT-II, the $\vec{Y}_{\text{long}}$ is effectively *spaced out*, with all its even-index terms set to zero. This is pretty convenient, because if we plug this in to the optimal interpolant equation, the DC and Nyquist terms disappear, leaving only the sum. This is a curious choice, but it [doesn't necessarily hurt performance](https://github.com/pavelkomarov/spectral-derivatives/blob/main/notebooks/dct_types.ipynb). In effect, instead of going up to wavenumber $N$, we pick every other wavenumber up to $2N$.

Once again, we don't have to be careful about our extension when fed to FFT versus IFFT, because the evenness of $Y_k = Y_{M-k}$ means it doesn't matter which one multiplies the positive versus the negative exponential ($e^{\pm jnk \frac{2\pi}{M}}$), which are flipped for forward versus inverse transform.

Notice this time there is no relationship with IDCT-II, which is the DCT-III, because the DCT-II is not its own inverse, and the DCT-III implies a different $\vec{Y}_{\text{long}}$.

This case is important for obtaining the frequency representation and for recovering even-order derivatives if we use the DCT-II as the forward transform.

In [46]:
def dct_ii_fft(Y):
	Y_ext = np.concatenate((Y, Y[::-1])) # even extension
	Y_spaced = [0 if i%2==0 else Y_ext[(i-1)//2] for i in range(4*len(Y))]
	return fft(Y_spaced)[:len(Y)]

def dct_ii_ifft(Y):
	Y_ext = np.concatenate((Y, Y[::-1])) # even extension
	Y_spaced = [0 if i%2==0 else Y_ext[(i-1)//2] for i in range(4*len(Y))]
	return len(Y_spaced)*ifft(Y_spaced)[:len(Y)]

assert np.all(np.abs(dct(Y_re, 2) - dct_ii_fft(Y_re)) < 1e-10)
assert np.all(np.abs(dct(Y_re, 2) - dct_ii_ifft(Y_re)) < 1e-10)

## DST-II

$$\begin{align}
\text{DST-II}([Y_0, Y_1, ... Y_N]) &= \text{FFT}([0, -Y_0, 0, -Y_1, 0, ... -Y_N, 0, Y_N, 0, Y_{N-1}, ... 0, Y_1])[1\!:\!N+2] \\
&= M \cdot \text{IFFT}([0, Y_0, 0, Y_1, 0, ... Y_N, 0, -Y_N, 0, -Y_{N-1}, ... 0, -Y_1])[1\!:\!N+2]
\end{align}
$$

Notice that for this case, as opposed to the DST-I case, we need not add extra $0$s to $\vec{Y}_{\text{long}}$. This is because the DST-II assumes oddness around half-indices above and below the ends of $\vec{Y}_{\text{short}}$, which correspond to the already-zeroed DC and Nyquist terms in $\vec{Y}_{\text{long}}$.

This case is important because $\text{IFFT}(\vec{Y}_{\text{long}})$ effectively occurs when taking odd-order derivatives with the DCT-II as the forward transform.

In [51]:
def dst_ii_fft(Y):
	Y_ext = np.concatenate((-Y, Y[::-1])) # odd extension, with sign flip to account for fft rather than ifft
	Y_spaced = [0 if i%2==0 else Y_ext[(i-1)//2] for i in range(4*len(Y))]
	return fft(Y_spaced)[1:1+len(Y)]

def dst_ii_ifft(Y):
	Y_ext = np.concatenate((Y, -Y[::-1])) # odd extension, this time with expected signs.
	Y_spaced = [0 if i%2==0 else Y_ext[(i-1)//2] for i in range(4*len(Y))]
	return len(Y_spaced)*ifft(Y_spaced)[1:1+len(Y)]

assert np.all(np.abs(dst(1j*Y_im, 2) - dst_ii_fft(Y_im)) < 1e-10)
assert np.all(np.abs(dst(1j*Y_im, 2) - dst_ii_ifft(Y_im)) < 1e-10)

In [111]:
y = np.array([3, 1, 7, 2])
N = len(y) - 1
M = 4*len(y)

# Forward and backward with DCT-II/III
Y_short = dct(y)
assert np.all(np.abs(dct(Y_short, 3)/(2*(N+1)) - y) < 1e-6)

# Forward and backward with FFT
y_ext = np.concatenate((y, y[::-1])) # even extension
y_spaced = np.array([0 if i%2==0 else y_ext[(i-1)//2] for i in range(M)])
Y_long = fft(y_spaced)
assert np.all(np.abs(Y_long[:len(Y_short)] - Y_short) < 1e-6)
assert np.all(np.abs(ifft(Y_long) - y_spaced) < 1e-6)


k = np.concatenate((np.arange(M//2 + 1), np.arange(-M//2 + 1, 0)))
k[M//2] = 0
print(k)

dY_long = (1j*k) * Y_long
print(Y_long)
print(dY_long)
print(ifft(dY_long))


[ 0  1  2  3  4  5  6  7  0 -7 -6 -5 -4 -3 -2 -1]
[ 26.        -0.j  -2.74444212+0.j  -4.24264069-0.j  11.85192125+0.j
   0.        +0.j -11.85192125+0.j   4.24264069+0.j   2.74444212+0.j
 -26.        -0.j   2.74444212-0.j   4.24264069-0.j -11.85192125-0.j
   0.        -0.j  11.85192125-0.j  -4.24264069+0.j  -2.74444212-0.j]
[ 0. +0.j         -0. -2.74444212j  0. -8.48528137j  0.+35.55576376j
  0. +0.j         -0.-59.25960627j  0.+25.45584412j  0.+19.21109486j
  0. -0.j         -0.-19.21109486j -0.-25.45584412j  0.+59.25960627j
 -0. +0.j         -0.-35.55576376j  0. +8.48528137j  0. +2.74444212j]
[  0.        +0.00000000e+00j   0.44974747-1.11022302e-16j
  -2.19731957+0.00000000e+00j  -4.53553391-1.11022302e-16j
  14.59636338+0.00000000e+00j  -1.53553391+1.11022302e-16j
 -10.68260094+0.00000000e+00j   3.44974747+1.11022302e-16j
   0.        +0.00000000e+00j  -3.44974747-1.11022302e-16j
  10.68260094+0.00000000e+00j   1.53553391-1.11022302e-16j
 -14.59636338+0.00000000e+00j   4.53553391